In [1]:
import time
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
class User:
    def __init__(self, path:str):
        if not path:
            self.id = ""
            self.password = ""
            self.keywords = []
            
        else:
            self.load_data(path)
            
    def load_data(self, path:str):
        data = open(path, 'r')
        self.id = data.readline()
        self.password = data.readline()
        self.keywords = list(data.readline().split())

In [3]:
options = webdriver.ChromeOptions() # Browser 세팅하기
options.add_argument('lang=ko_KR') # 사용언어 한국어
options.add_argument('disable-gpu') # 하드웨어 가속 안함
#options.add_argument('headless') # 창 숨기기

url = "https://portal.hanyang.ac.kr/sso/lgin.do"
user_data_path = "userdata.txt"

tm = time.localtime()
today = "{:04}.{:02}.{:02}".format(tm.tm_year, tm.tm_mon, tm.tm_mday)

In [4]:
today

'2023.08.03'

In [5]:
user = User(user_data_path)

In [6]:
driver = webdriver.Chrome(options=options)
driver.get(url)

In [7]:
driver.find_element(By.NAME, "userId").clear()
driver.find_element(By.NAME, "userId").send_keys(user.id)

driver.find_element(By.NAME, "password").clear()
driver.find_element(By.NAME, "password").send_keys(user.password)

try:
    driver.find_element(By.PARTIAL_LINK_TEXT, "로그인").click()
except:
    pass

In [8]:
driver.find_element(By.ID, "btn_cancel").click()

In [9]:
try:
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.dismiss() 
except:
    pass

In [10]:
try:
    driver.find_element(By.CLASS_NAME, 'more').click()
except:
    pass

In [11]:
try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btn_seoul")))
    driver.find_element(By.ID, "btn_seoul").click()
except:
    print("서울캠퍼스 공지 클릭 불가")

In [19]:
today_notices_xpath = f'//td[contains(text(), "{today}")]/preceding-sibling::td/span[@id="title"]'

page_xpath = [""] + [f'//*[@id="pagingPanel"]/span[{i}]/a[1]' for i in range(2, 12)]

In [16]:
try:
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="title"]')))
except:
    pass

In [20]:
notices = list()
for i in range(2, 11):
    today_notices_in_this_page = driver.find_elements(By.XPATH, today_notices_xpath)
    notices.extend(today_notices_in_this_page)
    if len(today_notices_in_this_page) <10:
        print(f"오늘 올라온 공지는 {len(notices)}개 입니다.")
        break
    driver.find_element(By.XPATH, page[i]).click()

오늘 올라온 공지는 7개 입니다.


In [21]:
print(*[notice.text for notice in notices], sep='\n')

[한국국제문화교류진흥원] 캄보디아 예술교육 역량 강화 연수 운영 대행 용역 입찰 공고
[DB손해보험] 대학생 서포터즈 DREAMER 12기 모집
[WRITING CENTER] 2023학년도 2학기 대학원생 장학조교 모집 안내
[성동구가족센터] 메타버스 심리상담 안내
[교육혁신팀] e-러닝 강좌 운영 보조 장학조교 모집 (~08.15)
[IC-PBL교수학습센터] 2023학년도 2학기 장학조교 모집 안내(3명) (~08/15)
[서울 공학교육혁신센터] 2023학년도 2학기 교육조교 모집 안내(FABLAB)


In [22]:
driver.quit()